In [3]:
import pandas as pd
import geopandas as gpd
import json


In [5]:
from tqdm import tqdm
import re


In [ ]:
import json
import shapely
import osmnx as ox

In [1]:
from DataAnalysis import DataPreparation as DataPreparation

In [ ]:
path_vol = "H:/Documents/ITMO/VKR/Data/Vologda"

In [8]:
# Загрузка исходного файла с ОКН
heritage_origin = pd.read_csv('Example_files/All_heritage.csv')

C:\Users\Way51\AppData\Local\Temp\ipykernel_20960\1288108848.py:2: DtypeWarning: Columns (1,50) have mixed types. Specify dtype option on import or set low_memory=False.
  heritage_origin = pd.read_csv('Example_files/All_heritage.csv')


In [9]:
# Выборка объектов в нужном регионе
REGION = 'Вологодская область'
ch_region, ch_region_coor, ch_region_none = DataPreparation.separator_reestr(heritage_origin, REGION) # Общий датасет, ГДФ(что можно найти), ДФ без координат оставшиеся

100%|██████████| 975/975 [00:00<00:00, 5027.63it/s]


0    Вологодская область
1    Вологодская область
2    Вологодская область
Name: Регион, dtype: object
Всего объектов: 975
Количество объектов с координатами: 292
Количество объектов без координат: 683


In [4]:
# Загрузка отчета о кадастровой оценке
report = gpd.read_file('Example_files/buildings_report.geojson') 
report = DataPreparation.number_report(report)

Skipping field Код объекта: unsupported OGR type: 10
h:\Documents\Python\Risks_of_investing_in_cultural_heritage\DataAnalysis.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '35-164339' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gdf.loc[i, 'учетный номер'] = desc


In [71]:
# УДАЛИТЬ
ch_region['Кадастровый номер'] = ''
for i in range(len(ch_region)):
    for j in range(len(report)):
        if str(ch_region.loc[i, 'Номер в реестре']) == str(report.loc[j, 'Номер в реестре']) or str(ch_region.loc[i, 'учетный номер']) == str(report.loc[j, 'учетный номер']):
            ch_region.loc[i, 'Кадастровый номер'] = str(ch_region.loc[i, 'Кадастровый номер']) + str(report.loc[j, 'Кадастровый номер'])

In [24]:
# Выборка необходимых столбцов для анализа
ch_region = ch_region[['Номер в реестре', 'учетный номер', 'Объект', 'Регион', 'Полный адрес', 'Категория историко-культурного значения', 'Вид объекта', 'описание предмета охраны', 'Принадлежность к Юнеско', 'наименование документа', 'building_year', 'lat', 'lon', 'geometry']]
report = report[['Номер в реестре','учетный номер', 'Кадастровый номер', 'Кадастровые номера ЗУ, в пределах которых расположен данный ОН', 'Кадастровые номера помещений в объекте недвижимости', 'Площадь, кв.м', 'Назначение здания', 'Год завершения строительства', 'Количество этажей (в том числе подземных)', 'Количество подземных этажей', 'Материал наружных стен', 'Класс конструктивной системы', 'Группа капитальности', 'Объект признан аварийным ', 'Функциональная группа', 'Подгруппа', 'usage', 'area', 'cad_cost', 'geometry']]

# Преобразование столбцов в нужный формат
ch_region['Номер в реестре'] = ch_region['Номер в реестре'].astype(str)
ch_region['учетный номер'] = ch_region['учетный номер'].astype(str)

In [79]:
# Разделение в зависимости от наличие информации
report_1 = report[report['Номер в реестре'] != 0]
report_2 = report[report['Номер в реестре'] == 0]

In [84]:
gdf_merged_1 = pd.merge(report_1, ch_region, on='Номер в реестре', how='inner')
gdf_merged_2 = pd.merge(report_2, ch_region, on='учетный номер', how='inner')

gdf_merged = pd.concat([gdf_merged_1, gdf_merged_2], ignore_index=True)
gdf_merged = gdf_merged.drop(['Номер в реестре_x', 'Номер в реестре_y', 'учетный номер_y', 'учетный номер_x'], axis = 1)


In [95]:
for i in range(len(gdf_merged)):
    if gdf_merged.loc[i, 'geometry_y'] != None:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_y'].centroid # Если геометрия не пустая, то из полигона делаем точку
    else:
        gdf_merged.loc[i, 'geometry_y'] = gdf_merged.loc[i, 'geometry_x']

gdf_merged = gpd.GeoDataFrame(gdf_merged,geometry='geometry_y', crs='EPSG:4326')

In [ ]:
cad_cost = pd.read_excel('Exmple_files/report_cost_1.xlsx')
cad_cost = cad_cost[['Кадастровый номер','Аварийность объекта','Экономическое устаревание', 'Эффективный возраст', 'Срок экономической жизни, лет', 'Физический износ', 'Совокупный износ','Удельный показатель кадаcтровой стоимости объекта недвижимости, руб./кв.м', 'Кадастровая стоимость (при оценке), руб']]
gdf_merged = pd.merge(gdf_merged, cad_cost, how='left', on='Кадастровый номер')

In [6]:
# Закружаем критерии оценки
with open("Example_files/criteria.json", encoding="UTF-8") as file_in:
    records = json.load(file_in)
print(records)

[{'spatial_aspect': {'public_transport': 0.5, 'public_transport_variable': 0.1, 'road': 0.3, 'improve_road': 0.1, 'cafe': 0.5, 'markets': 0.5, 'heritahe_nearby': 0.1, 'arch_style': 0.3, 'build_year': 0.3, 'build_floor': 0.3}, 'law_aspect': {'subject_of_protection': {'0': 1, '1': 1, '2': 0.75, '3': 0.75, '4': 0.25, '5': 0.25, '6': 0.25, '7': 0.0}, 'encumbrances': {'False': 1.0, 'True': 0}, 'permitted_land_use': {'buildings': 1}, 'cadastral_integrity': {'False': 1.0, 'True': 0}}, 'economic_aspect': {'cadstral_value_land': {'q1': 1, 'q2': 0.75, 'q3': 0.5, 'q4': 0.25}, 'cadstral_value_building': {'q1': 1, 'q2': 0.75, 'q3': 0.5, 'q4': 0.25}, 'support_measures': {'2': 1.0, '1': 0.5, '0': 0}}, 'physical_aspect': {'percentage_of_construction': {'20': 0.25, '40': 0.5, '60': 1.0, '80': 0.5, '90': 0.25, '100': 0.0}, 'material': {'name': {'unknown': 0.5, 'wood': 0.0, 'wood, stone': 0.5, 'stone': 1}}, 'function': {'industrial': 0, 'commercial': 0.5, 'living': 1.0}, 'using': {'this_moment': 1, 'less

In [12]:
719-551

168

In [19]:
report

,Кадастровый номер,Кадастровый квартал,Вид объекта недвижимости,Вид использования объекта недвижимости,"Площадь, кв.м",Дата присвоения кадастрового номера,Наименование объекта,Назначение сооружения,Назначение здания,Назначение помещения,...,Функциональная группа,Подгруппа,Заключение о достаточности информации о характеристиках ОН,usage,area,cad_cost,address,geometry,Номер в реестре,учетный номер
0,35:11:0403007:80,35:11:0403007,Здание,None,319.4,2011-11-24,Дом купца Никуличева,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",805,Собранной информации о характеристиках ОН дост...,Для обслуживания двухэтажного бревенчатого зда...,383.0,1279.22,"Вологодская область, р-н Усть-Кубинский, с Уст...","POLYGON ((39.72437 59.6272, 39.72433 59.627, 3...",351710909080005,0.0
1,35:24:0202005:636,35:24:0202005,Здание,None,355.3,2017-06-01,Нежилое здание,None,Нежилое,None,...,"Дома малоэтажной жилой застройки, в том числе ...",202,Собранной информации о характеристиках ОН дост...,Малоэтажная многоквартирная жилая застройка,1266.0,2360576.93,"Вологодская область, г Вологда, наб Пречистенс...","POLYGON ((39.90765 59.22112, 39.90693 59.22091...",0,35-164339
2,35:24:0201017:1222,35:24:0201017,Здание,None,233.5,2014-12-16,здание,None,Нежилое,None,...,Административные и бытовые объекты,603,Собранной информации о характеристиках ОН дост...,Малоэтажная многоквартирная жилая застройка,1281.0,2388545.85,"Вологодская область, г. Вологда, ул. Октябрьск...","POLYGON ((39.86866 59.221, 39.86842 59.22082, ...",351811311630006,0
3,35:03:0102058:129,35:03:0101034,Здание,None,489.1,2011-11-24,Церковь Иоанна Предтечи,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",816,Собранной информации о характеристиках ОН дост...,для эксплуатации и обслуживания памятника исто...,4149.0,13857.66,"Российская Федерация, Вологодская область, Бел...","POLYGON ((37.80068 60.03592, 37.80034 60.03612...",351610549810005,0
4,35:24:0202012:234,35:24:0202012,Здание,None,784.4,2014-02-12,Здание областной детской библиотеки,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",815,Собранной информации о характеристиках ОН дост...,ведение культурно-просветительской деятельности,1087.0,1530400.83,"Вологодская область, г. Вологда, пр-кт. Советс...","POLYGON ((39.89418 59.21756, 39.89388 59.21743...",351510309010006,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,35:14:0704027:63,35:14:0704027,Здание,None,150.7,2011-11-24,Церковь Иоанна Предтечи,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",816,Собранной информации о характеристиках ОН дост...,None,NaN,NaN,None,None,351610758520006,0
715,35:10:0103033:85,35:10:0103033,Здание,None,1220.6,2011-11-24,Церковь сретения,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",816,Собранной информации о характеристиках ОН дост...,None,NaN,NaN,None,None,351410171690006,0
716,35:10:0103033:86,35:10:0103033,Здание,None,23.3,2011-11-24,церковь николы гостинного,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",816,Собранной информации о характеристиках ОН дост...,Для обслуживания памятников архитектуры,8607.0,28747.38,"Вологодская область, р-н. Великоустюгский, г. ...","POLYGON ((46.30523 60.75733, 46.30385 60.75688...",351621269190006,0
717,35:10:0103033:88,35:10:0103033,Здание,None,589.7,2011-11-24,церковь николы гостинного,None,Нежилое,None,...,"Учебные, спортивные объекты, объекты культуры ...",816,Собранной информации о характеристиках ОН дост...,Для обслуживания памятников архитектуры,8607.0,28747.38,"Вологодская область, р-н. Великоустюгский, г. ...","POLYGON ((46.30523 60.75733, 46.30385 60.75688...",351621269190006,0
